<a href="https://colab.research.google.com/github/nicowilliamxvii/TQHDC_CS441/blob/main/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/cacfilecsvbaitaptqhdl/Canada-1.xlsx',
                       sheet_name='Canada by Citizenship',
                       skiprows=range(20),
)

In [ ]:
df

In [ ]:
print(df.shape)

In [ ]:
df.drop(['AREA', 'REG', 'DEV', 'Type', 'Coverage'], axis=1, inplace=False)

In [ ]:

df.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, inplace=True)
df.head()

In [ ]:

all(isinstance(column, str) for column in df.columns)
df.dtypes


In [ ]:
df.columns = list(map(str, df.columns))
all(isinstance(column, str) for column in df.columns)
df.dtypes

In [ ]:
df.set_index('Country', inplace=True)
df.head()

In [ ]:
df.columns = list(map(str, df.columns))
df.columns
years_int = list(range(1980, 2014)) # list years, type = int
years_str = list(map(str, years_int)) # Convert int to str
print(years_int)
print("=" * 280)
print(years_str)


In [ ]:
valid_years_str = [year for year in years_str if year in df.columns]
df[valid_years_str].head()

In [ ]:
df['Total'] = df[years_str].sum(axis=1)
df

In [ ]:
years = list(map(str, range(1980, 2015)))
years

In [ ]:
continent = df.groupby('Continent').sum()[['Total']]
continent

In [ ]:
# Giả sử các cột năm là tất cả các cột từ '1980' đến '2013'
year_columns = [str(year) for year in range(1980, 2014)]
continent_all_years = df.groupby('Continent')[year_columns].sum()
continent_all_years['All_Years_Sum'] = continent_all_years.sum(axis=1)
continent_all_years

In [ ]:
# Xem tổng quan
print("Tổng số nhập cư tất cả các năm theo châu lục:")
print(continent_all_years['All_Years_Sum'].sort_values(ascending=False))

# Tính phần trăm đóng góp của mỗi châu lục
total_immigrants = continent_all_years['All_Years_Sum'].sum()
continent_all_years['Percentage'] = (continent_all_years['All_Years_Sum'] / total_immigrants * 100).round(2)
print("\nPhần trăm đóng góp của mỗi châu lục:")
print(continent_all_years[['All_Years_Sum', 'Percentage']].sort_values('Percentage', ascending=False))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Biểu đồ cột - Tổng nhập cư theo châu lục
plt.figure(figsize=(10, 6))
continent_all_years['All_Years_Sum'].sort_values().plot(kind='barh')
plt.title('Tổng số nhập cư vào Canada theo châu lục (1980-2013)')
plt.xlabel('Số lượng nhập cư')
plt.ylabel('Châu lục')
plt.tight_layout()
plt.show()

# Biểu đồ đường - Xu hướng theo thời gian
plt.figure(figsize=(12, 6))
for continent in continent_all_years.index:
    if continent != 'World':  # Loại bỏ World để dễ nhìn
        plt.plot(year_columns, continent_all_years.loc[continent, year_columns], label=continent, marker='o')
plt.title('Xu hướng nhập cư vào Canada theo châu lục (1980-2013)')
plt.xlabel('Năm')
plt.ylabel('Số lượng nhập cư')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Tìm năm có số nhập cư cao nhất cho mỗi châu lục
max_year_by_continent = {}
for continent in continent_all_years.index:
    if continent != 'World':
        max_year = continent_all_years.loc[continent, year_columns].idxmax()
        max_value = continent_all_years.loc[continent, max_year]
        max_year_by_continent[continent] = (max_year, max_value)

print("Năm có số nhập cư cao nhất cho mỗi châu lục:")
for continent, (year, value) in max_year_by_continent.items():
    print(f"{continent}: {year} ({value:,} người)")

# Tính tốc độ tăng trưởng (2013 so với 1980)
growth_rates = {}
for continent in continent_all_years.index:
    if continent != 'World':
        start = continent_all_years.loc[continent, '1980']
        end = continent_all_years.loc[continent, '2013']
        growth = ((end - start) / start * 100) if start > 0 else 0
        growth_rates[continent] = growth

print("\nTốc độ tăng trưởng 1980-2013:")
for continent, growth in sorted(growth_rates.items(), key=lambda x: x[1], reverse=True):
    print(f"{continent}: {growth:.1f}%")

In [ ]:
# Chia thành các giai đoạn
periods = {
    '1980s': [str(year) for year in range(1980, 1990)],
    '1990s': [str(year) for year in range(1990, 2000)],
    '2000s': [str(year) for year in range(2000, 2010)],
    '2010-2013': [str(year) for year in range(2010, 2014)]
}

# Tính tổng cho mỗi giai đoạn
for period_name, years in periods.items():
    continent_all_years[f'Total_{period_name}'] = continent_all_years[years].sum(axis=1)

# Xem kết quả
print("Tổng nhập cư theo giai đoạn:")
period_columns = [f'Total_{period}' for period in periods.keys()]
print(continent_all_years[['All_Years_Sum'] + period_columns])

In [ ]:
# Tạo bảng xếp hạng
ranking_table = continent_all_years.copy()
ranking_table = ranking_table[ranking_table.index != 'World']  # Loại bỏ World

# Xếp hạng theo tổng số
ranking_table['Rank_Total'] = ranking_table['All_Years_Sum'].rank(ascending=False).astype(int)

# Xếp hạng theo năm 2013
ranking_table['Rank_2013'] = ranking_table['2013'].rank(ascending=False).astype(int)

# Xếp hạng theo tốc độ tăng trưởng
ranking_table['Growth_Rate'] = ((ranking_table['2013'] - ranking_table['1980']) / ranking_table['1980'] * 100)
ranking_table['Rank_Growth'] = ranking_table['Growth_Rate'].rank(ascending=False).astype(int)

print("Bảng xếp hạng:")
print(ranking_table[['All_Years_Sum', '2013', 'Growth_Rate', 'Rank_Total', 'Rank_2013', 'Rank_Growth']].sort_values('Rank_Total'))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6), sharey=True)

# Xếp hạng theo tổng số
ranking_total = ranking_table.sort_values('Rank_Total')
axes[0].barh(ranking_total.index, ranking_total['All_Years_Sum']/1000, color='skyblue')
axes[0].set_xlabel('Tổng số (nghìn người)')
axes[0].set_title('Xếp hạng theo tổng số (1980-2013)')
axes[0].invert_yaxis()  # Hạng 1 ở trên cùng

# Xếp hạng theo năm 2013
ranking_2013 = ranking_table.sort_values('Rank_2013')
axes[1].barh(ranking_2013.index, ranking_2013['2013']/1000, color='lightgreen')
axes[1].set_xlabel('Số lượng năm 2013 (nghìn)')
axes[1].set_title('Xếp hạng theo năm 2013')

# Xếp hạng theo tốc độ tăng trưởng
ranking_growth = ranking_table.sort_values('Rank_Growth')
axes[2].barh(ranking_growth.index, ranking_growth['Growth_Rate'], color='salmon')
axes[2].set_xlabel('Tốc độ tăng trưởng (%)')
axes[2].set_title('Xếp hạng theo tốc độ tăng trưởng')

# Thêm số hạng vào các cột
for i, (ax, rank_col) in enumerate(zip(axes, ['Rank_Total', 'Rank_2013', 'Rank_Growth'])):
    df = ranking_table.sort_values(rank_col)
    for j, (continent, row) in enumerate(df.iterrows()):
        if i == 0:
            value = f"{row['All_Years_Sum']/1000:,.1f}k"
        elif i == 1:
            value = f"{row['2013']/1000:,.1f}k"
        else:
            value = f"{row['Growth_Rate']:.1f}%"

        ax.text(row[['All_Years_Sum', '2013', 'Growth_Rate'][i]]/1000 if i!=2 else row['Growth_Rate'],
                j,
                f"#{int(row[rank_col])}",
                va='center', ha='left', fontweight='bold')

plt.tight_layout()
plt.show()